# Przetwarzanie Języka Naturalnego
## Zadanie 4
### Autor: Marcin Świątkowski

In [1]:
# !pip install -i https://pypi.clarin-pl.eu lpmn_client
import os
from bs4 import BeautifulSoup
from spacy.tokenizer import Tokenizer
from spacy.lang.pl import Polish
import math
from lpmn_client import download_file, upload_file
from lpmn_client import Task
import re
import json
import pandas as pd

2022-11-15 23:47:58.817397: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-15 23:47:58.817465: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-11-15 23:48:17.809304: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-11-15 23:48:17.809332: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-11-15 23:48:17.809358: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (marcin-Latitude-E7440): /proc/driver/nvidia/version does not exist


### Przydatne funkcje (w tym z poprzednich zadań):

In [232]:
# sprawdza czy w tekście jest kod html
def check_html(text):
    return bool(BeautifulSoup(text, "html.parser").find())

# usuwa za krótkie słowa oraz znaki niebędące literami
def reject_non_grata(dictionary, length=2):
    list_of_non_grata = "<>,.\";:'?/{}[]|\+=_-)(*&^%$#@!~`)'1234567890\n"
    new_dictionary = dict()
    for keys in dictionary.keys():
        good_keys = []
        for key in keys.split():
            if len(key) >= length and "  " not in key and not any(letter for letter in list(key) if letter in list_of_non_grata): # sprawdzenie długości i obecności nie-liter
                good_keys.append(key)
        if len(good_keys) == 2:
            merged_keys = ' '.join(good_keys)
            new_dictionary[merged_keys] = dictionary[merged_keys]
        
    return new_dictionary

# wariant powyższej dla tekstu zlematyzowanego i otagowanego
def lemma_reject_non_grata(dictionary, length=2):
    list_of_non_grata = r"<>,.\";'?/{}[]|\+=_-)(*&^%$#@!~`)'1234567890\n"
    new_dictionary = dict()
    for keys in dictionary.keys():
        good_keys = []
        for key in keys.split(" "):
            if len(key) >= length and "  " not in key and not any(letter for letter in list(key) if letter in list_of_non_grata): # sprawdzenie długości i obecności nie-liter
                if "::" not in key:
                    key_parts = key.split(":")
#                     print("key_parts: ", key_parts)
                    good_key_parts = []
                    for part in key_parts:
                        if len(part) >= length and "  " not in part and not any(letter for letter in list(part) if letter in list_of_non_grata): # sprawdzenie długości i obecności nie-liter
                            good_key_parts.append(part)
                    if len(good_key_parts) == 2:
                        merged_key = ':'.join(good_key_parts)
                        good_keys.append(merged_key)
        if len(good_keys) == 2:
#             print(good_keys)
            merged_keys = ' '.join(good_keys)
            new_dictionary[merged_keys] = dictionary[merged_keys]
    return new_dictionary

# robi to, co powyższa, ale dla jednego słowa
def reject_non_grata_one_word(dictionary, length=2):
    list_of_non_grata = "<>,.\";:'?/{}[]|\+=_-)(*&^%$#@!~`)'1234567890\n"
    new_dictionary = dict()
    for key in dictionary.keys():
        if len(key) >= length and "  " not in key and not any(letter for letter in list(key) if letter in list_of_non_grata): # sprawdzenie długości i obecności nie-liter
                new_dictionary[key] = dictionary[key]
    return new_dictionary

# wariant powyższej dla tekstu zlematyzowanego i otagowanego
def lema_reject_non_grata_one_word(dictionary, length=2):
    list_of_non_grata = "<>,.\";'?/{}[]|\+=_-)(*&^%$#@!~`)'1234567890\n"
    new_dictionary = dict()
    for key in dictionary.keys():
        if len(key) >= length and "  " not in key and "::" not in key and not any(letter for letter in list(key) if letter in list_of_non_grata): # sprawdzenie długości i obecności nie-liter
                new_dictionary[key] = dictionary[key]
    return new_dictionary

# oblicza częstość występowania wyrazów
def compute_freq(word_list, reject=True):
    word_frequency = dict()
    for word in word_list:
        if word in word_frequency.keys():
            word_frequency[word] += 1
        else:
            word_frequency[word] = 1
    
    if reject:
        return reject_non_grata(word_frequency)
    else:
        return word_frequency

# Funkcje zapisujące i ładujące
def save_list(list_to_save, filename, separator=','):
    with open(f'{filename}.txt','w') as f:
        f.write(f'{separator}'.join(list_to_save))

def load_list(filename, separator=','):
    with open(f'{filename}') as f:
        output = f.read().split(f'{separator}')
        return output

def save_dict(dict_to_save, filename):
    with open(f'{filename}.txt', 'w') as d:
        d.write(json.dumps(dict_to_save))

def load_dict(filename):
    with open(f'{filename}', 'r') as d:
        output = d.read()
        return output

In [6]:
# Ładowanie wcześniej przeprocesowanych plików:
list_of_all_words = load_list('/home/marcin/Pulpit/Studia semestr 2/NLP/lab2/outputs_lab4/list_of_all_words.txt')
bigrams_dict = load_dict('/home/marcin/Pulpit/Studia semestr 2/NLP/lab2/outputs_lab4/bigrams_dict.txt')
clean_all_words = load_dict('/home/marcin/Pulpit/Studia semestr 2/NLP/lab2/outputs_lab4/clean_all_words.txt')
clean_bigrams = load_dict('/home/marcin/Pulpit/Studia semestr 2/NLP/lab2/outputs_lab4/clean_bigrams.txt')
filtered_bigrams = load_dict('/home/marcin/Pulpit/Studia semestr 2/NLP/lab2/outputs_lab4/filtered_bigrams.txt')
all_words = load_dict('/home/marcin/Pulpit/Studia semestr 2/NLP/lab2/outputs_lab4/all_words.txt')

In [5]:
# Ładujemy korpus danych tekstowych:
path = r'/content/drive/MyDrive/Semestr zimowy 2022-23/NLP/LabyNLP/ustawy'
os.chdir(path)

In [25]:
# Stworzenie listy wszystkich słów w korpusie:
list_of_all_words = []


for file in os.listdir():
    if file.endswith(".txt"):
        file_path = f"{path}/{file}"
        
        with open(file_path, 'r', encoding='utf-8') as f:
            output = f.read()
            
            if check_html(output):
                print("Tekst zawiera kod html")
                break
            
            # Punkt 1: tokenizacja:
            nlp = Polish()
            doc = nlp(output)
            for token in doc:
#                 print(token.text)
                list_of_all_words.append(token.text.lower())
            break # dla jednego tekstu
            
print(list_of_all_words)

['\n\n\n', 'tekst', 'ustawy', '\n', 'ustalony', 'ostatecznie', 'po', 'rozpatrzeniu', 'poprawek', 'senatu', '\n\xa0\n\xa0\n\xa0\n', 'ustawa', '\n', 'z', 'dnia', '2', 'grudnia', '\n', '2009', 'r', '.', '\n\xa0\n', 'o', 'zmianie', 'ustawy', '–', '\n', 'prawo', 'zamówień', 'publicznych', '\n', 'oraz', 'niektórych', 'innych', 'ustaw[1', ']', ')', '\n\xa0\n\xa0\n', 'art', '.', '1', '.', '\n', 'w', 'ustawie', 'z', 'dnia', '\n', '29', 'stycznia', '2004', 'r', '.', '–', 'prawo', 'zamówień', 'publicznych', '(', 'dz', '.', '\xa0', 'u', '.', 'z', '2007', 'r', '.', 'nr', '223', ',', '\n', 'poz', '.', '1655', ',', 'z', 'późn', '.', 'zm.[2', ']', ')', ')', '\n', 'wprowadza', 'się', 'następujące', 'zmiany', ':', '\n', '1', ')', '\xa0\xa0 ', 'w', 'art', '.', '2', ':', '\n', 'a', ')', 'po', 'pkt', '7', 'dodaje', '\n', 'się', 'pkt', '7a', 'w', 'brzmieniu', ':', '\n', '„', '7a', ')', 'postępowaniu', '\n', 'o', 'udzielenie', 'zamówienia', '–', 'należy', 'przez', 'to', 'rozumieć', 'postępowanie', 'wszczynan

In [26]:
save_list(list_of_all_words, 'list_of_all_words')

## Punkt 1 i 2: tworzenie bigramów za pomocą tokenizatora

In [201]:
# Funkcja tworząca bigramy oraz listę słów, które się na nie składają

def compute_bigram(text_list, islist=True):
    if islist: # sprawdzamy czy input jest listą (przydatne w debugowaniu)
        text = ' '.join(text_list)
    else:
        text = text_list
    bigram_count = dict()
    all_words = []
    
    # Załączamy tokenizator:  
    nlp = Polish()
    doc = nlp(text)
    first_word = ""
    for token in doc:
        token = str(token.text).lower()
        all_words.append(token) # usupełniamy przy okazji listę wszystkich słów
        if len(first_word) > 0:
            bigram = first_word + " " + token
            if bigram in bigram_count.keys():
                bigram_count[bigram] += 1
            else:
                bigram_count[bigram] = 1
            first_word = token
        elif len(first_word) == 0:
            first_word = token
    
    return bigram_count, all_words

Przetestowanie funkcji tworzącej bigramy na przykładzie:

In [200]:
# przykład:
example = "The quick brown fox jumps over the lazy dog next to the other brown fox."
bigrams_dict, words = compute_bigram(example, False)
bigrams_dict

{'the quick': 1,
 'quick brown': 1,
 'brown fox': 2,
 'fox jumps': 1,
 'jumps over': 1,
 'over the': 1,
 'the lazy': 1,
 'lazy dog': 1,
 'dog next': 1,
 'next to': 1,
 'to the': 1,
 'the other': 1,
 'other brown': 1,
 'fox .': 1}

Tworzymy bigramy z korpusu:

In [206]:
bigrams_dict, words = compute_bigram(list_of_all_words)
save_list(words, 'words')
save_dict(bigrams_dict, 'bigrams_dict')
bigrams_dict

{'\n\n\n  tekst': 1,
 'tekst ustawy': 1,
 'ustawy \n ': 3,
 '\n  ustalony': 1,
 'ustalony ostatecznie': 1,
 'ostatecznie po': 1,
 'po rozpatrzeniu': 1,
 'rozpatrzeniu poprawek': 1,
 'poprawek senatu': 1,
 'senatu \n\xa0\n\xa0\n\xa0\n ': 1,
 '\n\xa0\n\xa0\n\xa0\n  ustawa': 1,
 'ustawa \n ': 1,
 '\n  z': 10,
 'z dnia': 19,
 'dnia 2': 1,
 '2 grudnia': 1,
 'grudnia \n ': 1,
 '\n  2009': 2,
 '2009 r': 7,
 'r .': 59,
 '. \n\xa0\n ': 9,
 '\n\xa0\n  o': 1,
 'o zmianie': 1,
 'zmianie ustawy': 1,
 'ustawy –': 1,
 '– \n ': 4,
 '\n  prawo': 1,
 'prawo zamówień': 2,
 'zamówień publicznych': 17,
 'publicznych \n ': 2,
 '\n  oraz': 11,
 'oraz niektórych': 1,
 'niektórych innych': 1,
 'innych ustaw[1': 1,
 'ustaw[1 ]': 1,
 '] )': 8,
 ') \n\xa0\n\xa0\n ': 1,
 '\n\xa0\n\xa0\n  art': 1,
 'art .': 199,
 '. 1': 83,
 '1 .': 48,
 '. \n ': 228,
 '\n  w': 29,
 'w ustawie': 6,
 'ustawie z': 4,
 'dnia \n ': 9,
 '\n  29': 2,
 '29 stycznia': 1,
 'stycznia 2004': 1,
 '2004 r': 8,
 '. –': 11,
 '– prawo': 2,
 'public

## Punkt 3: za pomocą funkcji reject_non_grata pozbywamy się par zawierających znaki niebędące literami oraz słowa krótsze niż 2

In [207]:
clean_bigrams = reject_non_grata(bigrams_dict)
save_dict(clean_bigrams, 'clean_bigrams')
clean_bigrams

{'tekst ustawy': 1,
 'ustalony ostatecznie': 1,
 'ostatecznie po': 1,
 'po rozpatrzeniu': 1,
 'rozpatrzeniu poprawek': 1,
 'poprawek senatu': 1,
 'zmianie ustawy': 1,
 'prawo zamówień': 2,
 'zamówień publicznych': 17,
 'oraz niektórych': 1,
 'niektórych innych': 1,
 'wprowadza się': 2,
 'się następujące': 3,
 'następujące zmiany': 2,
 'po pkt': 1,
 'się pkt': 6,
 'udzielenie zamówienia': 20,
 'należy przez': 2,
 'przez to': 2,
 'to rozumieć': 2,
 'rozumieć postępowanie': 1,
 'postępowanie wszczynane': 1,
 'drodze publicznego': 1,
 'publicznego ogłoszenia': 1,
 'zamówieniu lub': 4,
 'lub przesłania': 1,
 'przesłania zaproszenia': 1,
 'zaproszenia do': 5,
 'składania ofert': 17,
 'ofert albo': 1,
 'albo przesłania': 1,
 'do negocjacji': 2,
 'celu dokonania': 1,
 'wyboru oferty': 1,
 'oferty wykonawcy': 1,
 'którym zostanie': 1,
 'zostanie zawarta': 1,
 'zawarta umowa': 1,
 'sprawie zamówienia': 9,
 'przypadku trybu': 1,
 'trybu zamówienia': 3,
 'wolnej ręki': 13,
 'wynegocjowania postano

Obliczamy liczebność słów dla listy wszystkich słów, przy okazji oczyszczając ją za pomocą reject_non_grata:

In [211]:
all_words = compute_freq(words, False)
save_dict(all_words, 'all_words')
all_words

{'\n\n\n ': 4,
 'tekst': 1,
 'ustawy': 48,
 '\n ': 1371,
 'ustalony': 2,
 'ostatecznie': 1,
 'po': 50,
 'rozpatrzeniu': 1,
 'poprawek': 1,
 'senatu': 1,
 '\n\xa0\n\xa0\n\xa0\n ': 1,
 'ustawa': 5,
 'z': 202,
 'dnia': 66,
 '2': 128,
 'grudnia': 4,
 '2009': 7,
 'r': 59,
 '.': 1144,
 '\n\xa0\n ': 10,
 'o': 210,
 'zmianie': 1,
 '–': 55,
 'prawo': 4,
 'zamówień': 37,
 'publicznych': 24,
 'oraz': 65,
 'niektórych': 2,
 'innych': 6,
 'ustaw[1': 1,
 ']': 8,
 ')': 215,
 '\n\xa0\n\xa0\n ': 1,
 'art': 199,
 '1': 150,
 'w': 566,
 'ustawie': 7,
 '29': 6,
 'stycznia': 3,
 '2004': 10,
 '(': 16,
 'dz': 13,
 '\xa0 ': 141,
 'u': 8,
 '2007': 4,
 'nr': 155,
 '223': 1,
 '  \n ': 95,
 'poz': 156,
 '1655': 1,
 '  ': 751,
 'późn': 5,
 'zm.[2': 1,
 'wprowadza': 2,
 'się': 105,
 'następujące': 3,
 'zmiany': 10,
 ':': 114,
 '\xa0\xa0  ': 21,
 'a': 50,
 'pkt': 46,
 '7': 27,
 'dodaje': 17,
 '7a': 2,
 'brzmieniu': 18,
 '„': 60,
 'postępowaniu': 36,
 'udzielenie': 26,
 'zamówienia': 103,
 'należy': 3,
 'przez': 50,
 

In [212]:
clean_all_words = reject_non_grata_one_word(all_words)
save_dict(clean_all_words, 'clean_all_words')
clean_all_words

{'tekst': 1,
 'ustawy': 48,
 'ustalony': 2,
 'ostatecznie': 1,
 'po': 50,
 'rozpatrzeniu': 1,
 'poprawek': 1,
 'senatu': 1,
 'ustawa': 5,
 'dnia': 66,
 'grudnia': 4,
 'zmianie': 1,
 'prawo': 4,
 'zamówień': 37,
 'publicznych': 24,
 'oraz': 65,
 'niektórych': 2,
 'innych': 6,
 'art': 199,
 'ustawie': 7,
 'stycznia': 3,
 'dz': 13,
 '\xa0 ': 141,
 'nr': 155,
 'poz': 156,
 'późn': 5,
 'wprowadza': 2,
 'się': 105,
 'następujące': 3,
 'zmiany': 10,
 'pkt': 46,
 'dodaje': 17,
 'brzmieniu': 18,
 'postępowaniu': 36,
 'udzielenie': 26,
 'zamówienia': 103,
 'należy': 3,
 'przez': 50,
 'to': 9,
 'rozumieć': 2,
 'postępowanie': 22,
 'wszczynane': 1,
 'drodze': 8,
 'publicznego': 20,
 'ogłoszenia': 53,
 'zamówieniu': 24,
 'lub': 150,
 'przesłania': 6,
 'zaproszenia': 5,
 'do': 141,
 'składania': 22,
 'ofert': 24,
 'albo': 54,
 'negocjacji': 19,
 'celu': 6,
 'dokonania': 3,
 'wyboru': 10,
 'oferty': 24,
 'wykonawcy': 25,
 'którym': 28,
 'zostanie': 3,
 'zawarta': 10,
 'umowa': 13,
 'sprawie': 19,
 'p

## Punkt 4: obliczamy PMI dla naszego korpusu

In [225]:
def compute_PMI(bigrams_dict, all_words_dict):
    PMI_count = dict()
    total_words = sum(all_words_dict.values())
    for bigram in bigrams_dict:
        pair = bigram.split()
        try:
            w1_count = all_words_dict[pair[0]]
        except KeyError:
            print("error: ", pair)
            break
        try:
            w2_count = all_words_dict[pair[1]]
        except IndexError:
            continue
        Pw1 = w1_count/total_words
        Pw2 = w2_count/total_words
        Pww = bigrams_dict[bigram]/total_words
        
        PMI = math.log2(Pww/(Pw1*Pw2))
        PMI_count[bigram] = PMI
    return PMI_count

In [286]:
PMIs = compute_PMI(clean_bigrams, clean_all_words)
# save_dict(PMIs, 'PMIs')
PMIs

{'tekst ustawy': 7.439138279531754,
 'ustalony ostatecznie': 12.02410078025291,
 'ostatecznie po': 7.380244590478185,
 'po rozpatrzeniu': 7.380244590478185,
 'rozpatrzeniu poprawek': 13.02410078025291,
 'poprawek senatu': 13.02410078025291,
 'zmianie ustawy': 7.439138279531754,
 'prawo zamówień': 6.81464741462396,
 'zamówień publicznych': 7.317147755153144,
 'oraz niektórych': 6.001732967224456,
 'niektórych innych': 9.439138279531754,
 'wprowadza się': 6.309855262586788,
 'się następujące': 6.309855262586788,
 'następujące zmiany': 9.117210184644392,
 'po pkt': 1.8566826344211722,
 'się pkt': 3.3712558072509307,
 'udzielenie zamówienia': 5.9590886298159615,
 'należy przez': 6.79528208975703,
 'przez to': 5.210319589035873,
 'to rozumieć': 9.854175778810598,
 'rozumieć postępowanie': 7.564669161615613,
 'postępowanie wszczynane': 8.564669161615612,
 'drodze publicznego': 5.7021726853655474,
 'publicznego ogłoszenia': 2.9742522308023482,
 'zamówieniu lub': 3.210319589035873,
 'lub przes

## Punkt 5: sortujemy obliczone PMI malejąco i wyświetlamy top 10 pozycji

In [289]:
sorted_PMIs = {k: v for k, v in sorted(PMIs.items(), key=lambda item: item[1], reverse=True)}
PMIs_iter = iter(sorted_PMIs.items())
bigram_PMIs = []
for _ in range(10):
    bigram_PMIs.append(next(PMIs_iter))
for PMI in bigram_PMIs:
    print(PMI)

('rozpatrzeniu poprawek', 13.02410078025291)
('poprawek senatu', 13.02410078025291)
('wszelkie świadczenia', 13.02410078025291)
('usługami określonymi', 13.02410078025291)
('dofinansowanie projektu', 13.02410078025291)
('uczciwej konkurencji', 13.02410078025291)
('badań naukowych', 13.02410078025291)
('prac rozwojowych', 13.02410078025291)
('koprodukcja materiałów', 13.02410078025291)
('materiałów programowych', 13.02410078025291)


## Punkt 6: filtrujemy obliczone bigramy tak, że pozostają te, które występują więcej niż 5 razy

In [231]:
filtered_bigrams = {k: v for k, v in clean_bigrams.items() if v >= 5}
filtered_bigrams = {k: v for k, v in sorted(filtered_bigrams.items(), key=lambda item: item[1], reverse=True)}
save_dict(filtered_bigrams, 'filtered_bigrams')

sorted_filtered_bigrams = {k: v for k, v in sorted(filtered_bigrams.items(), key=lambda item: item[1], reverse=True)}
filtered_bigrams_iter = iter(sorted_filtered_bigrams.items())
for _ in range(10):
    print(next(filtered_bigrams_iter))

('na podstawie', 44)
('otrzymuje brzmienie', 34)
('od dnia', 33)
('podstawie art', 32)
('dni od', 30)
('stosuje się', 25)
('wydanych na', 24)
('kwoty określone', 23)
('którym mowa', 22)
('których mowa', 21)


### Obliczamy PMI po przefiltrowaniu

In [320]:
PMIs = compute_PMI(filtered_bigrams, clean_all_words)
# save_dict(PMIs, 'PMIs')
PMIs

{'na podstawie': 5.802733364317226,
 'otrzymuje brzmienie': 7.504233307753639,
 'od dnia': 6.166119785125338,
 'podstawie art': 4.772766315594053,
 'dni od': 6.458300536618649,
 'stosuje się': 6.253271734220419,
 'wydanych na': 5.8425343723752015,
 'kwoty określone': 7.996915950926679,
 'którym mowa': 7.032321287057878,
 'których mowa': 6.487159794394698,
 'udzielenie zamówienia': 5.9590886298159615,
 'przepisach wydanych': 7.793146345413038,
 'postępowania odwoławczego': 7.171657968666768,
 'zamówień publicznych': 7.317147755153144,
 'składania ofert': 8.067169502144797,
 'dodaje się': 6.222392421336448,
 'zawarcia umowy': 6.699598741701721,
 'wolnej ręki': 9.323661062111817,
 'negocjacji bez': 8.569722389341898,
 'zamówienia jest': 4.453077470489628,
 'wniesienia odwołania': 6.509007498648346,
 'bez ogłoszenia': 6.974252230802348,
 'wartość zamówienia': 5.7526377523485355,
 'zamówienia publicznego': 5.600634658903485,
 'prezes izby': 6.046820856752993,
 'przepisów ustawy': 6.37500794

In [321]:
sorted_PMIs = {k: v for k, v in sorted(PMIs.items(), key=lambda item: item[1], reverse=True)}
PMIs_iter = iter(sorted_PMIs.items())
bigram_PMIs = []
for _ in range(10):
    bigram_PMIs.append(next(PMIs_iter))
for PMI in bigram_PMIs:
    print(PMI)

('dzienniku urzędowym', 10.439138279531754)
('roboty budowlane', 10.17610387369796)
('unii europejskiej', 10.02410078025291)
('posiedzeniu niejawnym', 10.02410078025291)
('urzędowym unii', 9.953711452361512)
('przetargu nieograniczonego', 9.854175778810598)
('wspólnot europejskich', 9.564669161615612)
('urzędowi oficjalnych', 9.564669161615612)
('składu orzekającego', 9.550169591920499)
('karę finansową', 9.427165637865677)


## Punkt 7: tagowanie i lematyzacja korpusu za pomocą API Clarin-PL

In [78]:
task = Task(lpmn='any2txt|morphoditasgjp')
task.email = "martinswiat@mail.com"

file_id = upload_file("/content/drive/MyDrive/Semestr zimowy 2022-23/NLP/LabyNLP/ustawy.zip")
output_file_id = task.run(file_id)
download_file(output_file_id, "out")

'out/a067612f-bc93-464e-989e-c614beab6921.zip'

## Punkt 8: obliczanie bigramów dla zlematyzowanego i otagowanego korpusu

In [7]:
# Funkcja parsujące ze zlematyzowanego tekstu w formacie xml
def lemme_parse(text_line):
    get_base_pattern = "(?<=\<base>).+(?=</base>)"
    get_tag_pattern =  "(?<=\<ctag>).+(?=(</ctag>))"
    
    base = re.search(get_base_pattern, text_line).group()
    tag = re.search(get_tag_pattern, text_line).group()
    if ":" in tag:
        tag = tag.split(':')[0]
    
    return base, tag

# Funkcja zwracająca sparsowane bazę i tag w postaci "baza:tag"
def make_word_tagged_again(text_line):
    lem, tag = lemme_parse(text_line)
    bigram = lem + ":" + tag
    return bigram

# Funkcja, która zapisuje tagowane słowa (bigramy) do przygotowanej listy
def collect_tagged_words(tagged_word):
    global lemme_tagged_list
    lemme_tagged_list.append(tagged_word)

# Funkcja obliczająca liczebność bigramów dla zlematyzowanego korpusu
def compute_bigrams_for_lema(tagged_list):
    all_tagged_words = []
    lema_bigram_count = dict()
    first_word = ""
    for token in tagged_list:
        token = str(token).lower()
        all_tagged_words.append(token)
        if len(first_word) > 0:
            bigram = first_word + " " + token
            if bigram in lema_bigram_count.keys():
                lema_bigram_count[bigram] += 1
            else:
                lema_bigram_count[bigram] = 1
            first_word = token
        elif len(first_word) == 0:
            first_word = token
    
    return lema_bigram_count, all_tagged_words

In [23]:
# Inicjalizacja listy lematów z tagami oraz słownika bigramów po lematyzacji
# URUCHOMIĆ ŻEBY ZRESETOWAĆ

lemme_tagged_list = []
lemme_bigrams = dict()

In [25]:
# Wczytujemy zlematyzowany korpus:
path = r'/home/marcin/Pulpit/Studia semestr 2/NLP/lab2/outputs_lab4/corpus_after_lemma'
os.chdir(path)

for file in os.listdir():
    if file.endswith(".txt"):
        file_path = f"{path}/{file}"
        
        with open(file_path, 'r', encoding='utf-8') as f:
            lines = [line.rstrip() for line in f]
            for line in lines:
                try:
                    make_word_tagged_again(line)
                    collect_tagged_words(make_word_tagged_again(line))
                except AttributeError: # na wypadek linii, które zwracają None
                    continue

In [26]:
# save_list(lemme_tagged_list, '/home/marcin/Pulpit/Studia semestr 2/NLP/lab2/outputs_lab4/lemme_tagged_list')

In [27]:
lemme_tagged_list = load_list('/home/marcin/Pulpit/Studia semestr 2/NLP/lab2/outputs_lab4/lemme_tagged_list.txt')

Poniżej wariant funkcji reject_non_grata, dostosowana tak, żeby usuwając dwukropki nie usunęła tych, które powinny zostać

In [151]:
# def lemma_reject_non_grata(dictionary, length=2):
#     list_of_non_grata = r"<>,.\";'?/{}[]|\+=_-)(*&^%$#@!~`)'1234567890\n"
#     new_dictionary = dict()
#     for keys in dictionary.keys():
#         good_keys = []
#         for key in keys.split(" "):
#             if len(key) >= length and "  " not in key and not any(letter for letter in list(key) if letter in list_of_non_grata): # sprawdzenie długości i obecności nie-liter
#                 if "::" not in key:
#                     key_parts = key.split(":")
# #                     print("key_parts: ", key_parts)
#                     good_key_parts = []
#                     for part in key_parts:
#                         if len(part) >= length and "  " not in part and not any(letter for letter in list(part) if letter in list_of_non_grata): # sprawdzenie długości i obecności nie-liter
#                             good_key_parts.append(part)
#                     if len(good_key_parts) == 2:
#                         merged_key = ':'.join(good_key_parts)
#                         good_keys.append(merged_key)
#         if len(good_keys) == 2:
# #             print(good_keys)
#             merged_keys = ' '.join(good_keys)
#             new_dictionary[merged_keys] = dictionary[merged_keys]
#     return new_dictionary

## Punkt 9: tworzenie i liczenie bigramów ze zlematyzowanego korpusu

In [99]:
lemmatized_bigrams, lemmatized_list = compute_bigrams_for_lema(lemme_tagged_list)
save_dict(lemmatized_bigrams, '/home/marcin/Pulpit/Studia semestr 2/NLP/lab2/outputs_lab4/lemmatized_bigrams')
save_list(lemmatized_list, '/home/marcin/Pulpit/Studia semestr 2/NLP/lab2/outputs_lab4/lemmatized_list')
lemmatized_bigrams

{'dziennik:brev .:interp': 9287,
 '.:interp ustawa:brev': 6210,
 'ustawa:brev .:interp': 6210,
 '.:interp z:prep': 5912,
 'z:prep rok:brev': 19760,
 'rok:brev .:interp': 35454,
 '.:interp numer:brev': 23085,
 'numer:brev ': 44963,
 ':interp pozycja:brev': 46664,
 'pozycja:brev .:interp': 48797,
 '.:interp ustawa:subst': 4074,
 'ustawa:subst z:prep': 9229,
 'z:prep dzień:subst': 12139,
 'dzień:subst styczeń:subst': 1923,
 'styczeń:subst rok:brev': 1740,
 '.:interp o:prep': 8334,
 'o:prep zmiana:subst': 1502,
 'zmiana:subst ustawa:subst': 973,
 'ustawa:subst o:prep': 1794,
 'o:prep gospodarka:subst': 157,
 'gospodarka:subst nieruchomość:subst': 105,
 'nieruchomość:subst oraz:conj': 60,
 'oraz:conj inny:adj': 1004,
 'inny:adj ustawa:subst': 323,
 'ustawa:subst artykuł:brev': 111,
 'artykuł:brev .:interp': 89891,
 '.:interp .:interp': 103235,
 '.:interp w:prep': 22128,
 'w:prep ustawa:subst': 5660,
 'dzień:subst sierpień:subst': 1133,
 'sierpień:subst rok:brev': 1104,
 'nieruchomość:subst 

### Oczyszczenie z niechcianych znaków i wyrazów:

In [153]:
clean_lemmatized_bigrams = lemma_reject_non_grata(lemmatized_bigrams)
save_dict(clean_lemmatized_bigrams, '/home/marcin/Pulpit/Studia semestr 2/NLP/lab2/outputs_lab4/clean_lemmatized_bigrams')
clean_lemmatized_bigrams

{'dzień:subst styczeń:subst': 1923,
 'styczeń:subst rok:brev': 1740,
 'ustawa:subst artykuł:brev': 111,
 'dzień:subst sierpień:subst': 1133,
 'sierpień:subst rok:brev': 1104,
 'dzień:subst czerwiec:subst': 1553,
 'czerwiec:subst rok:brev': 1458,
 'zakwaterować:ger siła:subst': 32,
 'rzeczpospolita:subst polski:adj': 4105,
 'dzień:subst maj:subst': 855,
 'maj:subst rok:brev': 771,
 'skarb:subst państwo:subst': 1917,
 'przez:prep to:subst': 1122,
 'który:adj wielkość:subst': 7,
 'dostęp:subst do:prep': 470,
 'do:prep droga:subst': 29,
 'wymóg:subst realizacja:subst': 2,
 'realizacja:subst obiekt:subst': 2,
 'akt:subst prawo:subst': 140,
 'prawo:subst miejscowy:adj': 143,
 'samorząd:subst województwo:subst': 318,
 'prawo:subst powiat:subst': 219,
 'także:part podział:subst': 4,
 'rozumieć:ger przepis:subst': 550,
 'były:adj hitlerowski:adj': 9,
 'hitlerowski:adj obóz:subst': 9,
 'obóz:subst zagłada:subst': 13,
 'miejsce:subst pamięć:subst': 5,
 'siedlisko:subst przyroda:subst': 3,
 'po:pr

Przefiltrowanie

In [348]:
lema_filtered_bigrams = {k: v for k, v in clean_lemmatized_bigrams.items() if v >= 5}
lema_filtered_bigrams = {k: v for k, v in sorted(lema_filtered_bigrams.items(), key=lambda item: item[1], reverse=True)}
lema_filtered_bigrams

{'który:adj mowa:subst': 30662,
 'do:prep sprawa:subst': 9347,
 'właściwy:adj do:prep': 9163,
 'od:prep dzień:subst': 5719,
 'rzeczpospolita:subst polski:adj': 4105,
 'dzień:subst od:prep': 2717,
 'przepis:subst artykuł:brev': 2712,
 'do:prep dzień:subst': 2599,
 'co:prep mało:adv': 2471,
 'dzień:subst wejść:ger': 2382,
 'prawo:subst do:prep': 2372,
 'dzień:subst grudzień:subst': 2111,
 'grudzień:subst rok:brev': 2037,
 'przepis:subst ustawa:subst': 1969,
 'dzień:subst styczeń:subst': 1923,
 'skarb:subst państwo:subst': 1917,
 'po:prep upływ:subst': 1746,
 'styczeń:subst rok:brev': 1740,
 'miesiąc:subst od:prep': 1684,
 'po:prep ustęp:brev': 1639,
 'po:prep wyraz:subst': 1637,
 'przepis:subst ustęp:brev': 1631,
 'zwać:ppas daleko:adv': 1590,
 'podstawa:subst artykuł:brev': 1589,
 'do:prep obrót:subst': 1586,
 'dzień:subst czerwiec:subst': 1553,
 'przed:prep dzień:subst': 1549,
 'po:prep artykuł:brev': 1497,
 'czerwiec:subst rok:brev': 1458,
 'dzień:subst lipiec:subst': 1406,
 'zasada:s

## Punkt 10: obliczenie PMI i wyświetlenie 10 najczęstszych par

Tutaj niestety nie udało mi się tego zrobić do końca, ponieważ po drodze wyskakuje błąd związany z niezgodnością słownika bigramów i słownika wszystkich słów

In [162]:
# Ta funkcja jest dostosowana do specyfiki bigramów ze zlematyzowanego korpusu

def compute_PMI_lema(bigrams_dict, all_words_dict):
    PMI_count = dict()
    total_words = sum(all_words_dict.values())
    for bigram in bigrams_dict:
        semipair = bigram.split()
        nested_pair = [w.split(":") for w in semipair]
        pair = [item for sublist in nested_pair for item in sublist][0::2]
#         print("nested pair: ",nested_pair)
#         print("pair: ",pair)
#         print("semipair: ",semipair)
        try:
            w1_count = all_words_dict[pair[0]]
#             print(w1_count)
        except KeyError:
#             print(pair)
            continue
        w2_count = all_words_dict[pair[1]]
        Pw1 = w1_count/total_words
        Pw2 = w2_count/total_words
        Pww = bigrams_dict[bigram]/total_words
        
        PMI = math.log2(Pww/(Pw1*Pw2))
        PMI_count[bigram] = PMI
    return PMI_count

# def lema_reject_non_grata_one_word(dictionary, length=2):
#     list_of_non_grata = "<>,.\";'?/{}[]|\+=_-)(*&^%$#@!~`)'1234567890\n"
#     new_dictionary = dict()
#     for key in dictionary.keys():
#         if len(key) >= length and "  " not in key and "::" not in key and not any(letter for letter in list(key) if letter in list_of_non_grata): # sprawdzenie długości i obecności nie-liter
#                 new_dictionary[key] = dictionary[key]
#     return new_dictionary

In [326]:
lema_bigrams_PMIs = compute_PMI(lema_filtered_bigrams, lema_reject_non_grata_one_word(compute_freq(lemmatized_list,False)))
save_dict(lema_bigrams_PMIs, "/home/marcin/Pulpit/Studia semestr 2/NLP/lab2/outputs_lab4/lema_bigrams_PMIs")
lema_bigrams_PMIs

{'który:adj mowa:subst': 6.307857327960423,
 'do:prep sprawa:subst': 5.097914620117145,
 'właściwy:adj do:prep': 5.3060095372158695,
 'od:prep dzień:subst': 5.519536356610709,
 'rzeczpospolita:subst polski:adj': 8.863015886792288,
 'dzień:subst od:prep': 4.445788016629396,
 'przepis:subst artykuł:brev': 2.759067329475755,
 'do:prep dzień:subst': 2.5048871822901093,
 'co:prep mało:adv': 10.371637339849537,
 'dzień:subst wejść:ger': 6.955491332591229,
 'prawo:subst do:prep': 3.776633767033179,
 'dzień:subst grudzień:subst': 7.02870934445233,
 'grudzień:subst rok:brev': 6.723750013350334,
 'przepis:subst ustawa:subst': 4.0539535739859085,
 'dzień:subst styczeń:subst': 7.071619924319019,
 'skarb:subst państwo:subst': 9.062978654630648,
 'po:prep upływ:subst': 7.433596132150313,
 'styczeń:subst rok:brev': 6.673869734994484,
 'miesiąc:subst od:prep': 6.326567937793495,
 'po:prep ustęp:brev': 3.0587919068345513,
 'po:prep wyraz:subst': 5.171325436051268,
 'przepis:subst ustęp:brev': 2.6744200

Wyświetlenie top 10

In [349]:
sorted_lema_bi_PMIs = {k: v for k, v in sorted(lema_bigrams_PMIs.items(), key=lambda item: item[1], reverse=True)}
PMIs_iter = iter(sorted_lema_bi_PMIs.items())
lema_bigram_PMIs = []
for _ in range(10):
    lema_bigram_PMIs.append(next(PMIs_iter))
for PMI in lema_bigram_PMIs:
    print(PMI)

('chrom:subst sześciowartościowy:adj', 19.42999816675898)
('adam:subst mickiewicz:subst', 19.01496066748014)
('zgrzewarka:subst oporowy:adj', 18.751926261646343)
('maria:subst curie:subst', 18.693032572592774)
('hugo:subst kołłątaj:subst', 18.693032572592774)
('środa:subst wielkopolski:brev', 18.693032572592774)
('lampa:subst wyładowczy:adj', 18.693032572592774)
('jedwab:subst wiskozowy:adj', 18.55552904884284)
('michał:subst oczapowski:subst', 18.42999816675898)
('zatoka:subst gdański:adj', 18.330462493208067)


## Punkt 11: robimy to samo, ale dla trigramów

In [185]:
# Poniżej funkcja obliczająca trigramy, wedle tej samej zasady, co bigramy, ale w większej ilości kroków

def compute_trigram(text_list, islist=True):
    if islist:
        text = ' '.join(text_list)
    else:
        text = text_list # to dla testów, gdy wprowadzałem zdanie w postaci stringa
    trigram_count = dict()    
    all_words = []
    nlp = Polish()
    doc = nlp(text)
    first_word = ""
    second_word = ""
    for token in doc:
        token = str(token.text).lower()
        all_words.append(token)
        if len(first_word) > 0:
            trigram = first_word + " " + second_word + " " + token
            
            if trigram in trigram_count.keys():
                trigram_count[trigram] += 1
            else:
                trigram_count[trigram] = 1
                
            first_word = second_word
            second_word = token
        elif len(first_word) == 0 and len(second_word) > 0:
            first_word = second_word
            second_word = token
        elif len(first_word) == 0 and len(second_word) == 0:
            second_word = token
            
    return trigram_count, all_words

Test na próbkowym tekście

In [252]:
example = "The quick brown fox jumps over the lazy dog next to the other brown fox. The quick brown"
trigram_count, all_words_tri = compute_trigram(example, False)
trigram_count

{'the quick brown': 2,
 'quick brown fox': 1,
 'brown fox jumps': 1,
 'fox jumps over': 1,
 'jumps over the': 1,
 'over the lazy': 1,
 'the lazy dog': 1,
 'lazy dog next': 1,
 'dog next to': 1,
 'next to the': 1,
 'to the other': 1,
 'the other brown': 1,
 'other brown fox': 1,
 'brown fox .': 1,
 'fox . the': 1,
 '. the quick': 1}

Teraz na korpusie właściwym:

In [292]:
example = "The quick brown fox jumps over the lazy dog next to the other brown fox. The quick brown"
trigram_count, all_words_tri = compute_trigram(list_of_all_words)
trigram_count

{'\n\n\n  tekst ustawy': 1,
 'tekst ustawy \n ': 1,
 'ustawy \n  ustalony': 1,
 '\n  ustalony ostatecznie': 1,
 'ustalony ostatecznie po': 1,
 'ostatecznie po rozpatrzeniu': 1,
 'po rozpatrzeniu poprawek': 1,
 'rozpatrzeniu poprawek senatu': 1,
 'poprawek senatu \n\xa0\n\xa0\n\xa0\n ': 1,
 'senatu \n\xa0\n\xa0\n\xa0\n  ustawa': 1,
 '\n\xa0\n\xa0\n\xa0\n  ustawa \n ': 1,
 'ustawa \n  z': 1,
 '\n  z dnia': 1,
 'z dnia 2': 1,
 'dnia 2 grudnia': 1,
 '2 grudnia \n ': 1,
 'grudnia \n  2009': 1,
 '\n  2009 r': 2,
 '2009 r .': 7,
 'r . \n\xa0\n ': 1,
 '. \n\xa0\n  o': 1,
 '\n\xa0\n  o zmianie': 1,
 'o zmianie ustawy': 1,
 'zmianie ustawy –': 1,
 'ustawy – \n ': 1,
 '– \n  prawo': 1,
 '\n  prawo zamówień': 1,
 'prawo zamówień publicznych': 2,
 'zamówień publicznych \n ': 2,
 'publicznych \n  oraz': 1,
 '\n  oraz niektórych': 1,
 'oraz niektórych innych': 1,
 'niektórych innych ustaw[1': 1,
 'innych ustaw[1 ]': 1,
 'ustaw[1 ] )': 1,
 '] ) \n\xa0\n\xa0\n ': 1,
 ') \n\xa0\n\xa0\n  art': 1,
 '\n\xa

### Punkt 11 AKA 3: usuwamy niechciane słowa i znaki

In [245]:
# Wariant funkcji reject_non_grata dla trigramów
def reject_trigram(dictionary, length=2):
    list_of_non_grata = "<>,.\";:'?/{}[]|\+=_-)(*&^%$#@!~`)'1234567890\n"
    new_dictionary = dict()
    for keys in dictionary.keys():
        good_keys = []
        for key in keys.split():
            if len(key) >= length and "  " not in key and not any(letter for letter in list(key) if letter in list_of_non_grata): # sprawdzenie długości i obecności nie-liter
                good_keys.append(key)
        if len(good_keys) == 3: # właściwie jedyna różnica jest tutaj
            merged_keys = ' '.join(good_keys)
            new_dictionary[merged_keys] = dictionary[merged_keys]
    return new_dictionary

In [293]:
trigram_clean = reject_trigram(trigram_count)
trigram_clean

{'ustalony ostatecznie po': 1,
 'ostatecznie po rozpatrzeniu': 1,
 'po rozpatrzeniu poprawek': 1,
 'rozpatrzeniu poprawek senatu': 1,
 'prawo zamówień publicznych': 2,
 'oraz niektórych innych': 1,
 'wprowadza się następujące': 2,
 'się następujące zmiany': 2,
 'należy przez to': 2,
 'przez to rozumieć': 2,
 'to rozumieć postępowanie': 1,
 'rozumieć postępowanie wszczynane': 1,
 'drodze publicznego ogłoszenia': 1,
 'zamówieniu lub przesłania': 1,
 'lub przesłania zaproszenia': 1,
 'przesłania zaproszenia do': 1,
 'składania ofert albo': 1,
 'ofert albo przesłania': 1,
 'zaproszenia do negocjacji': 1,
 'wyboru oferty wykonawcy': 1,
 'którym zostanie zawarta': 1,
 'zostanie zawarta umowa': 1,
 'przypadku trybu zamówienia': 1,
 'wynegocjowania postanowień takiej': 1,
 'postanowień takiej umowy': 1,
 'to rozumieć wszelkie': 1,
 'rozumieć wszelkie świadczenia': 1,
 'których przedmiotem nie': 1,
 'przedmiotem nie są': 1,
 'roboty budowlane lub': 3,
 'budowlane lub dostawy': 1,
 'są usługami 

Obliczamy liczbę wszystkich słów w korpusie

In [294]:
all_words_tri_freq = reject_non_grata_one_word(compute_freq(all_words_tri, False))
all_words_tri_freq

{'tekst': 1,
 'ustawy': 48,
 'ustalony': 2,
 'ostatecznie': 1,
 'po': 50,
 'rozpatrzeniu': 1,
 'poprawek': 1,
 'senatu': 1,
 'ustawa': 5,
 'dnia': 66,
 'grudnia': 4,
 'zmianie': 1,
 'prawo': 4,
 'zamówień': 37,
 'publicznych': 24,
 'oraz': 65,
 'niektórych': 2,
 'innych': 6,
 'art': 199,
 'ustawie': 7,
 'stycznia': 3,
 'dz': 13,
 '\xa0 ': 141,
 'nr': 155,
 'poz': 156,
 'późn': 5,
 'wprowadza': 2,
 'się': 105,
 'następujące': 3,
 'zmiany': 10,
 'pkt': 46,
 'dodaje': 17,
 'brzmieniu': 18,
 'postępowaniu': 36,
 'udzielenie': 26,
 'zamówienia': 103,
 'należy': 3,
 'przez': 50,
 'to': 9,
 'rozumieć': 2,
 'postępowanie': 22,
 'wszczynane': 1,
 'drodze': 8,
 'publicznego': 20,
 'ogłoszenia': 53,
 'zamówieniu': 24,
 'lub': 150,
 'przesłania': 6,
 'zaproszenia': 5,
 'do': 141,
 'składania': 22,
 'ofert': 24,
 'albo': 54,
 'negocjacji': 19,
 'celu': 6,
 'dokonania': 3,
 'wyboru': 10,
 'oferty': 24,
 'wykonawcy': 25,
 'którym': 28,
 'zostanie': 3,
 'zawarta': 10,
 'umowa': 13,
 'sprawie': 19,
 'p

## Punkt 12: obliczamy PMI dla trigramów

In [248]:
def compute_PMI_trigram(trigrams_dict, all_words_dict):
    PMI_count = dict()
    total_words = sum(all_words_dict.values())
    for trigram in trigrams_dict:
        trio = trigram.split()
        try:
            w1_count = all_words_dict[trio[0]]
        except KeyError:
            print("error: ", trio)
            break
        w2_count = all_words_dict[trio[1]]
        w3_count = all_words_dict[trio[2]]
        Pw1 = w1_count/total_words
        Pw2 = w2_count/total_words
        Pw3 = w3_count/total_words
        Pww = trigrams_dict[trigram]/total_words
        
        PMI = math.log2(Pww/(Pw1*Pw2*Pw3))
        PMI_count[trigram] = PMI
    return PMI_count

### Filtrujemy przed obliczeniem PMI żeby wykluczyć, te słowa, które występują mniej niż 5 razy

In [295]:
filtered_trigrams = {k: v for k, v in trigram_clean.items() if v >= 1}
filtered_trigrams = {k: v for k, v in sorted(filtered_trigrams.items(), key=lambda item: item[1], reverse=True)}
filtered_trigrams

{'na podstawie art': 29,
 'dni od dnia': 25,
 'wydanych na podstawie': 21,
 'przepisach wydanych na': 16,
 'negocjacji bez ogłoszenia': 12,
 'wartość zamówienia jest': 12,
 'niż kwoty określone': 11,
 'dodaje się ust': 9,
 'istotnych warunków zamówienia': 9,
 'biuletynie zamówień publicznych': 9,
 'urzędowi oficjalnych publikacji': 9,
 'jest równa lub': 9,
 'równa lub przekracza': 9,
 'lub przekracza kwoty': 9,
 'nie stosuje się': 8,
 'stosuje się odpowiednio': 8,
 'przekracza kwoty określone': 8,
 'do wniesienia odwołania': 8,
 'jest mniejsza niż': 7,
 'zamówienia jest równa': 7,
 'terminu do wniesienia': 7,
 'sprawie zamówienia publicznego': 7,
 'trybie negocjacji bez': 6,
 'specyfikacji istotnych warunków': 6,
 'na stronie internetowej': 6,
 'zamówieniu urzędowi oficjalnych': 6,
 'mniejsza niż kwoty': 6,
 'kodeks postępowania cywilnego': 6,
 'przed dniem wejścia': 6,
 'termin składania ofert': 5,
 'zamówienia jest mniejsza': 5,
 'składania ofert nie': 5,
 'publikacji wspólnot europe

In [296]:
tri_PMIs = compute_PMI_trigram(filtered_trigrams, all_words_tri_freq)
tri_PMIs

{'na podstawie art': 10.588758900516762,
 'dni od dnia': 13.174972791679311,
 'wydanych na podstawie': 13.059280230570508,
 'przepisach wydanych na': 13.581232933765863,
 'negocjacji bez ogłoszenia': 15.750425497611673,
 'wartość zamówienia jest': 13.19177603188029,
 'niż kwoty określone': 14.564568970981114,
 'dodaje się ust': 11.041378771684746,
 'istotnych warunków zamówienia': 14.859200692793419,
 'biuletynie zamówień publicznych': 16.101782600710663,
 'urzędowi oficjalnych publikacji': 18.34084242842494,
 'jest równa lub': 13.082417275843733,
 'równa lub przekracza': 15.345451681677527,
 'lub przekracza kwoty': 14.023523586790164,
 'nie stosuje się': 10.947015797515387,
 'stosuje się odpowiednio': 13.546053483448265,
 'przekracza kwoty określone': 16.175526680235215,
 'do wniesienia odwołania': 11.69311720836137,
 'jest mniejsza niż': 14.878276559063508,
 'zamówienia jest równa': 13.262165359771688,
 'terminu do wniesienia': 12.334462178980045,
 'sprawie zamówienia publicznego': 1

### Sortujemy nasze wyniki i wyświetlamy top 10

In [297]:
sorted_tri_PMIs = {k: v for k, v in sorted(tri_PMIs.items(), key=lambda item: item[1], reverse=True)}
PMIs_iter = iter(sorted_tri_PMIs.items())
trigram_PMIs = []
for _ in range(10):
    trigram_PMIs.append(next(PMIs_iter))
for PMI in trigram_PMIs:
    print(PMI)

('rozpatrzeniu poprawek senatu', 26.04820156050582)
('koprodukcja materiałów programowych', 26.04820156050582)
('materiałów programowych przeznaczonych', 26.04820156050582)
('dokumentów potwierdzających spełnianie', 26.04820156050582)
('terminowej realizacji zadań', 26.04820156050582)
('powodowałaby konieczność nabywania', 26.04820156050582)
('nieproporcjonalnie duże trudności', 26.04820156050582)
('duże trudności techniczne', 26.04820156050582)
('potrzeby własne jednostki', 26.04820156050582)
('własne jednostki wojskowej', 26.04820156050582)


### Obliczamy trigramy oraz PMI dla zlematyzowanego korpusu

In [305]:
# Funkcja obliczająca liczebność trigramów dla zlematyzowanego korpusu
def compute_trigrams_for_lema(tagged_list):
    all_tagged_words = []
    lema_trigram_count = dict()
    first_word = ""
    second_word = ""
    for token in tagged_list:
        token = str(token).lower()
        all_tagged_words.append(token)
        if len(first_word) > 0:
            trigram = first_word + " " + second_word + " " + token
            if trigram in lema_trigram_count.keys():
                lema_trigram_count[trigram] += 1
            else:
                lema_trigram_count[trigram] = 1
            first_word = second_word
            second_word = token
        elif len(first_word) == 0 and len(second_word) > 0:
            first_word = second_word
            second_word = token
        elif len(first_word) == 0 and len(second_word) == 0:
            second_word = token
    
    return lema_trigram_count, all_tagged_words

# Poniżej dwie funkcje będące wariantem reject_non_grata dla trigramów z korpusu zlematyzowanego i otagowanego

def lema_reject_trigram(dictionary, length=2):
    list_of_non_grata = r"<>,.\";'?/{}[]|\+=_-)(*&^%$#@!~`)'1234567890\n"
    new_dictionary = dict()
    for keys in dictionary.keys():
        good_keys = []
        for key in keys.split(" "):
            if len(key) >= length and "  " not in key and not any(letter for letter in list(key) if letter in list_of_non_grata): # sprawdzenie długości i obecności nie-liter
                if "::" not in key:
                    key_parts = key.split(":")
#                     print("key_parts: ", key_parts)
                    good_key_parts = []
                    for part in key_parts:
                        if len(part) >= length and "  " not in part and not any(letter for letter in list(part) if letter in list_of_non_grata): # sprawdzenie długości i obecności nie-liter
                            good_key_parts.append(part)
                    if len(good_key_parts) == 2:
                        merged_key = ':'.join(good_key_parts)
                        good_keys.append(merged_key)
        if len(good_keys) == 3:
#             print(good_keys)
            merged_keys = ' '.join(good_keys)
            new_dictionary[merged_keys] = dictionary[merged_keys]
    return new_dictionary

def lema_reject_non_grata_one_word(dictionary, length=2):
    list_of_non_grata = "<>,.\";'?/{}[]|\+=_-)(*&^%$#@!~`)'1234567890\n"
    new_dictionary = dict()
    for key in dictionary.keys():
        if len(key) >= length and "  " not in key and "::" not in key and not any(letter for letter in list(key) if letter in list_of_non_grata): # sprawdzenie długości i obecności nie-liter
                new_dictionary[key] = dictionary[key]
    return new_dictionary

In [300]:
lemme_tagged_list

['dziennik:brev',
 '.:interp',
 'ustawa:brev',
 '.:interp',
 'z:prep',
 'rok:brev',
 '.:interp',
 'numer:brev',
 '',
 ':interp',
 'pozycja:brev',
 '.:interp',
 'ustawa:subst',
 'z:prep',
 'dzień:subst',
 'styczeń:subst',
 'rok:brev',
 '.:interp',
 'o:prep',
 'zmiana:subst',
 'ustawa:subst',
 'o:prep',
 'gospodarka:subst',
 'nieruchomość:subst',
 'oraz:conj',
 'inny:adj',
 'ustawa:subst',
 'artykuł:brev',
 '.:interp',
 '.:interp',
 'w:prep',
 'ustawa:subst',
 'z:prep',
 'dzień:subst',
 'sierpień:subst',
 'rok:brev',
 '.:interp',
 'o:prep',
 'gospodarka:subst',
 'nieruchomość:subst',
 '(:interp',
 'dziennik:brev',
 '.:interp',
 'ustawa:brev',
 '.:interp',
 'numer:brev',
 '',
 ':interp',
 'pozycja:brev',
 '.:interp',
 '741:num',
 '',
 ':interp',
 'z:prep',
 'rok:brev',
 '.:interp',
 'numer:brev',
 '',
 ':interp',
 'pozycja:brev',
 '.:interp',
 '668:num',
 'oraz:conj',
 'z:prep',
 'rok:brev',
 '.:interp',
 'numer:brev',
 '',
 ':interp',
 'pozycja:brev',
 '.:interp',
 '484:num',
 'i:conj',


In [301]:
lema_trigram_count, lema_all_words_tri = compute_trigrams_for_lema(lemme_tagged_list)
lema_trigram_count

{'dziennik:brev .:interp ustawa:brev': 6210,
 '.:interp ustawa:brev .:interp': 6210,
 'ustawa:brev .:interp z:prep': 2960,
 '.:interp z:prep rok:brev': 4297,
 'z:prep rok:brev .:interp': 19741,
 'rok:brev .:interp numer:brev': 19281,
 '.:interp numer:brev ': 22327,
 'numer:brev  :interp': 44963,
 ':interp pozycja:brev .:interp': 46644,
 'pozycja:brev .:interp ustawa:subst': 348,
 '.:interp ustawa:subst z:prep': 1278,
 'ustawa:subst z:prep dzień:subst': 9193,
 'z:prep dzień:subst styczeń:subst': 1135,
 'dzień:subst styczeń:subst rok:brev': 1675,
 'styczeń:subst rok:brev .:interp': 1739,
 'rok:brev .:interp o:prep': 7606,
 '.:interp o:prep zmiana:subst': 925,
 'o:prep zmiana:subst ustawa:subst': 928,
 'zmiana:subst ustawa:subst o:prep': 697,
 'ustawa:subst o:prep gospodarka:subst': 25,
 'o:prep gospodarka:subst nieruchomość:subst': 95,
 'gospodarka:subst nieruchomość:subst oraz:conj': 10,
 'nieruchomość:subst oraz:conj inny:adj': 3,
 'oraz:conj inny:adj ustawa:subst': 17,
 'inny:adj usta

In [302]:
lema_trigram_clean = lema_reject_trigram(lema_trigram_count)
lema_trigram_clean

{'dzień:subst styczeń:subst rok:brev': 1675,
 'dzień:subst sierpień:subst rok:brev': 1095,
 'dzień:subst czerwiec:subst rok:brev': 1415,
 'dzień:subst maj:subst rok:brev': 764,
 'dostęp:subst do:prep droga:subst': 9,
 'wymóg:subst realizacja:subst obiekt:subst': 2,
 'akt:subst prawo:subst miejscowy:adj': 130,
 'były:adj hitlerowski:adj obóz:subst': 9,
 'hitlerowski:adj obóz:subst zagłada:subst': 9,
 'chyba:part że:comp przepis:subst': 66,
 'że:comp przepis:subst dotyczyć:pact': 3,
 'przepis:subst dotyczyć:pact gospodarować:ger': 5,
 'między:prep skarb:subst państwo:subst': 6,
 'jeżeli:comp przepis:subst ustawa:subst': 37,
 'udzielić:ger zgoda:subst przez:prep': 4,
 'zgoda:subst przez:prep rada:subst': 2,
 'droga:subst uchwała:subst rada:subst': 12,
 'od:prep wpływ:subst osiągać:ppas': 2,
 'tytuł:subst trwały:adj zarząd:subst': 3,
 'od:prep dzień:subst wywiesić:ger': 4,
 'dzień:subst wywiesić:ger wykaz:subst': 4,
 'przed:prep dzień:subst grudzień:subst': 19,
 'dzień:subst grudzień:subst

In [304]:
lema_all_words_tri

['dziennik:brev',
 '.:interp',
 'ustawa:brev',
 '.:interp',
 'z:prep',
 'rok:brev',
 '.:interp',
 'numer:brev',
 '',
 ':interp',
 'pozycja:brev',
 '.:interp',
 'ustawa:subst',
 'z:prep',
 'dzień:subst',
 'styczeń:subst',
 'rok:brev',
 '.:interp',
 'o:prep',
 'zmiana:subst',
 'ustawa:subst',
 'o:prep',
 'gospodarka:subst',
 'nieruchomość:subst',
 'oraz:conj',
 'inny:adj',
 'ustawa:subst',
 'artykuł:brev',
 '.:interp',
 '.:interp',
 'w:prep',
 'ustawa:subst',
 'z:prep',
 'dzień:subst',
 'sierpień:subst',
 'rok:brev',
 '.:interp',
 'o:prep',
 'gospodarka:subst',
 'nieruchomość:subst',
 '(:interp',
 'dziennik:brev',
 '.:interp',
 'ustawa:brev',
 '.:interp',
 'numer:brev',
 '',
 ':interp',
 'pozycja:brev',
 '.:interp',
 '741:num',
 '',
 ':interp',
 'z:prep',
 'rok:brev',
 '.:interp',
 'numer:brev',
 '',
 ':interp',
 'pozycja:brev',
 '.:interp',
 '668:num',
 'oraz:conj',
 'z:prep',
 'rok:brev',
 '.:interp',
 'numer:brev',
 '',
 ':interp',
 'pozycja:brev',
 '.:interp',
 '484:num',
 'i:conj',


In [306]:
lema_all_words_tri_freq = lema_reject_non_grata_one_word(compute_freq(lema_all_words_tri, False))
lema_all_words_tri_freq

{'dziennik:brev': 9287,
 'ustawa:brev': 6211,
 'z:prep': 94225,
 'rok:brev': 35543,
 'numer:brev': 48525,
 ':interp': 369772,
 'pozycja:brev': 48821,
 'ustawa:subst': 26603,
 'dzień:subst': 29816,
 'styczeń:subst': 2032,
 'o:prep': 69647,
 'zmiana:subst': 7656,
 'gospodarka:subst': 3142,
 'nieruchomość:subst': 2377,
 'oraz:conj': 36120,
 'inny:adj': 13022,
 'artykuł:brev': 89903,
 'w:prep': 217366,
 'sierpień:subst': 1152,
 'i:conj': 96944,
 'wprowadzać:fin': 2155,
 'się:ign': 49393,
 'następujący:adj': 2922,
 'punkt:brev': 20432,
 'wyraz:subst': 13242,
 'jednostka:subst': 8947,
 'zastępować:fin': 5215,
 'kropka:subst': 901,
 'przecinek:subst': 984,
 'dodawać:fin': 9090,
 'brzmienie:subst': 19182,
 'czerwiec:subst': 1624,
 'zakwaterować:ger': 44,
 'siła:subst': 663,
 'zbrojny:adj': 537,
 'rzeczpospolita:subst': 4372,
 'polski:adj': 8546,
 'maj:subst': 881,
 'gospodarować:ger': 347,
 'niektóry:adj': 1198,
 'składnik:subst': 1058,
 'mienie:subst': 930,
 'skarb:subst': 1924,
 'państwo:sub

In [307]:
lema_filtered_trigrams = {k: v for k, v in lema_trigram_clean.items() if v >= 1}
lema_filtered_trigrams = {k: v for k, v in sorted(lema_filtered_trigrams.items(), key=lambda item: item[1], reverse=True)}
lema_filtered_trigrams

{'właściwy:adj do:prep sprawa:subst': 8554,
 'dzień:subst od:prep dzień:subst': 2265,
 'dzień:subst grudzień:subst rok:brev': 1912,
 'dzień:subst styczeń:subst rok:brev': 1675,
 'dzień:subst czerwiec:subst rok:brev': 1415,
 'miesiąc:subst od:prep dzień:subst': 1377,
 'terytorium:subst rzeczpospolita:subst polski:adj': 1343,
 'dzień:subst lipiec:subst rok:brev': 1300,
 'dzień:subst sierpień:subst rok:brev': 1095,
 'do:prep sprawa:subst gospodarka:subst': 1067,
 'ustawa:subst zostać:praet ogłosić:ppas': 1013,
 'od:prep dzień:subst ogłosić:ger': 1003,
 'do:prep sprawa:subst zdrowie:subst': 977,
 'dzień:subst kwiecień:subst rok:brev': 937,
 'dzień:subst wrzesień:subst rok:brev': 916,
 'dzień:subst listopad:subst rok:brev': 892,
 'życie:subst po:prep upływ:subst': 868,
 'do:prep sprawa:subst środowisko:subst': 792,
 'od:prep dzień:subst wejść:ger': 781,
 'upływ:subst dzień:subst od:prep': 775,
 'dzień:subst maj:subst rok:brev': 764,
 'przed:prep dzień:subst wejść:ger': 761,
 'po:prep upływ:

In [308]:
lema_tri_PMIs = compute_PMI_trigram(lema_filtered_trigrams, lema_all_words_tri_freq)
lema_tri_PMIs

{'właściwy:adj do:prep sprawa:subst': 13.104430966559491,
 'dzień:subst od:prep dzień:subst': 11.334446065092306,
 'dzień:subst grudzień:subst rok:brev': 13.783548120116482,
 'dzień:subst styczeń:subst rok:brev': 13.770105088242177,
 'dzień:subst czerwiec:subst rok:brev': 13.850094815315915,
 'miesiąc:subst od:prep dzień:subst': 13.18736592261335,
 'terytorium:subst rzeczpospolita:subst polski:adj': 18.277368950262534,
 'dzień:subst lipiec:subst rok:brev': 13.882376134779772,
 'dzień:subst sierpień:subst rok:brev': 13.97563454782807,
 'do:prep sprawa:subst gospodarka:subst': 12.36446460454712,
 'ustawa:subst zostać:praet ogłosić:ppas': 16.692421552346328,
 'od:prep dzień:subst ogłosić:ger': 14.062329729660682,
 'do:prep sprawa:subst zdrowie:subst': 12.265151214470492,
 'dzień:subst kwiecień:subst rok:brev': 13.870901082990471,
 'dzień:subst wrzesień:subst rok:brev': 13.862908620600281,
 'dzień:subst listopad:subst rok:brev': 13.875329704896624,
 'życie:subst po:prep upływ:subst': 15.98

In [352]:
sorted_lema_tri_PMIs = {k: v for k, v in sorted(lema_tri_PMIs.items(), key=lambda item: item[1], reverse=True)}
PMIs_iter = iter(sorted_lema_tri_PMIs.items())
lema_trigram_PMIs = []
for _ in range(10):
    lema_trigram_PMIs.append(next(PMIs_iter))
for PMI in lema_trigram_PMIs:
    print(PMI)

('dosuszać:ger blaszka:subst liściowy:adj', 42.02992133496028)
('chleb:subst świętojański:adj strąkowy:adj', 42.02992133496028)
('tematyka:subst szeroko:adv pojąć:ppas', 40.44495883423912)
('karboksylowy:adj kok:subst liście:subst', 40.44495883423912)
('grecja:subst peseta:subst hiszpański:adj', 39.70799324007291)
('biało:adv emaliować:ppas otok:subst', 39.63760391218151)
('tarczowy:adj cyrkularka:subst szlifierka:subst', 39.02992133496028)
('cyrkularka:subst szlifierka:subst tokarka:subst', 39.02992133496028)
('cezary:subst grabarczyk:subst wicemarszałek:subst', 38.85999633351796)
('elżbieta:subst radziszewska:subst wicemarszałek:subst', 38.85999633351796)


## Punkt 13: porównanie rezultatów

# WAŻNE: Wszystkie obliczenia zostały przeprowadzone na tylko części zbioru. Musiałem tak zrobić z powodu ograniczeń sprzętowych. Przy załadowaniu i tokenizacji całego korpusu wyświetlił mi się następujący błąd:
IOPub data rate exceeded.

The notebook server will temporarily stop sending output

to the client in order to avoid crashing it.
# W związku z tym uruchomiłem cały niezlematyzowany kod na części zbioru i przetestowałem działanie funkcji. Ich funkcjonalność wygląda na dobrą, ale wyniki mogą się różnić o tych, które możnaby uzyskać na całym zbiorze

Dla bigramów:

In [351]:
bigram_words = [w1 for w1, w2 in bigram_PMIs]
bigram_values = [w2 for w1, w2 in bigram_PMIs]
lema_bigram_words = [w1 for w1, w2 in lema_bigram_PMIs]
lema_bigram_values = [w2 for w1, w2 in lema_bigram_PMIs]

In [346]:
d_bigrams = {"Bigram PMIs words": bigram_words, 
             "Bigram values": bigram_values, 
             "Lemmatized Bigram PMIs words": lema_bigram_words, 
             "Lemmatized Bigram values": lema_bigram_values}

df_bi = pd.DataFrame(d_bigrams, columns=["Bigram PMIs words", "Bigram values", "Lemmatized Bigram PMIs words", "Lemmatized Bigram values"])
df_bi

,Bigram PMIs words,Bigram values,Lemmatized Bigram PMIs words,Lemmatized Bigram values
0,dzienniku urzędowym,10.439138,chrom:subst sześciowartościowy:adj,19.429998
1,roboty budowlane,10.176104,adam:subst mickiewicz:subst,19.014961
2,unii europejskiej,10.024101,zgrzewarka:subst oporowy:adj,18.751926
3,posiedzeniu niejawnym,10.024101,maria:subst curie:subst,18.693033
4,urzędowym unii,9.953711,hugo:subst kołłątaj:subst,18.693033
5,przetargu nieograniczonego,9.854176,środa:subst wielkopolski:brev,18.693033
6,wspólnot europejskich,9.564669,lampa:subst wyładowczy:adj,18.693033
7,urzędowi oficjalnych,9.564669,jedwab:subst wiskozowy:adj,18.555529
8,składu orzekającego,9.550170,michał:subst oczapowski:subst,18.429998
9,karę finansową,9.427166,zatoka:subst gdański:adj,18.330462


Dla trigramów:

In [353]:
trigram_words = [w1 for w1, w2 in trigram_PMIs]
trigram_values = [w2 for w1, w2 in trigram_PMIs]
lema_trigram_words = [w1 for w1, w2 in lema_trigram_PMIs]
lema_trigram_values = [w2 for w1, w2 in lema_trigram_PMIs]

In [354]:
d_trigrams = {"Trigram PMIs words": trigram_words, 
             "Trigram values": trigram_values, 
             "Lemmatized Trigram PMIs words": lema_trigram_words, 
             "Lemmatized Trigram values": lema_trigram_values}

df_tri = pd.DataFrame(d_trigrams, columns=["Trigram PMIs words", "Trigram values", "Lemmatized Trigram PMIs words", "Lemmatized Trigram values"])
df_tri

,Trigram PMIs words,Trigram values,Lemmatized Trigram PMIs words,Lemmatized Trigram values
0,rozpatrzeniu poprawek senatu,26.048202,dosuszać:ger blaszka:subst liściowy:adj,42.029921
1,koprodukcja materiałów programowych,26.048202,chleb:subst świętojański:adj strąkowy:adj,42.029921
2,materiałów programowych przeznaczonych,26.048202,tematyka:subst szeroko:adv pojąć:ppas,40.444959
3,dokumentów potwierdzających spełnianie,26.048202,karboksylowy:adj kok:subst liście:subst,40.444959
4,terminowej realizacji zadań,26.048202,grecja:subst peseta:subst hiszpański:adj,39.707993
5,powodowałaby konieczność nabywania,26.048202,biało:adv emaliować:ppas otok:subst,39.637604
6,nieproporcjonalnie duże trudności,26.048202,tarczowy:adj cyrkularka:subst szlifierka:subst,39.029921
7,duże trudności techniczne,26.048202,cyrkularka:subst szlifierka:subst tokarka:subst,39.029921
8,potrzeby własne jednostki,26.048202,cezary:subst grabarczyk:subst wicemarszałek:subst,38.859996
9,własne jednostki wojskowej,26.048202,elżbieta:subst radziszewska:subst wicemarszałe...,38.859996


## Punkt 14: odpowiedź na pytania

### Pytanie 1: Why do we have to filter the bigrams, rather than the token sequence?
Ponieważ przefiltrowanie przed utwaorzeniem bigramów mogłoby nam stworzyć coś w rodzaju "fantomowych par". Na przykład: mając zdanie "Jacek poszedł. Kasia została" jedna z par wyglądałaby na "poszedł Kasia" gdybyśmy przed utworzeniem bigramów usunęli kropkę.

### Pytanie 2: Which method works better for the bigrams and which for the trigrams?
Jeśli nie popełniłem błędu w kodzie (a mam nadzieję, że tak nie jest) to można odnieść wrażenie, że w PMI ze zlematyzowanego tekstu zwróciło bardziej "sensowne" wyniki dla bigramów niż dla trigramów. Możnaby więc powiedzieć, że lematyzacja i tagowanie niezbyt sprawdzają się w przypadku trigramów, ale da się je zastosować dla bigramów z w miarę zadowalającym skutkiem. W przypadku teksty niezlematyzowanego i nieotagowanego to w obu przypadkach widzimy dosyć sensowne wyniki, aczkolwiek wydaje się, że lepiej to wygląda dla trigramów.

### Pytanie 3: What types of expressions are discovered by the methods.
Dla zlematyzowanego i otagowanego korpusu da zauważalna jest obecność nazwisk, tak dla bigramów jak i dla trigramów. Natomiast tekst niezlematyzowany zwrócił dla bigramów i trigramów więcej wyrażeń o charakterze prawniczym.

### Pytanie 4: Can you devise a different type of filtering that would yield better results?
Przy ulepszeniu wyników zwracanych dla korpusu zlematyzowanego i otagowanego możnaby wziąć pod uwagę tagi. Chodziłoby o to, by wykluczyć takie trójki, które składają się z części mowy, które zwykle w języku nie występują ze sobą.
Ewentualnie, gdyby powyższe miało okazać się bez sensu, możnaby spróbować filtrować wyniki w oparciu o tagi nie tyle wykluczając statystycznie rzadkie zbitki, ale używając tagów do bardziej precyzyjnego wyszukiwania tego, co chcemy (np. terminologii prawniczej, a nie cyrkularek, szlifierek, tokarek...)